# Potential Data Discretization

The goal of this notebook is to further explore options and develop a method/ function to discretize the possible frequency ranges to build targets for a deep learning/ other models.

## Things to fix
- Drop ranges that are "Impossible"- ranges that have not been observed at
- probably convert to numpy for stuff instead of base pytho 
- return a matrix instead of a list/ vector of probabilities (this may not be necessary)

In [92]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import plotly.express as px

measurements = pd.read_csv('./nrao_measurements.csv')

In [93]:
measurements

,project_code,project_title,project_abstract,fs_type,low_freq,high_freq,target,diff_freq,med_freq,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text
0,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.38,90.62,1,0.24,90.500,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
1,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.70,90.93,1,0.23,90.815,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,91.69,91.92,1,0.23,91.805,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
3,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,92.89,93.12,1,0.23,93.005,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
4,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,217.59,218.53,1,0.94,218.060,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67430,2023.A.00009.S,Finding t=0: Tracing the Origins of Rocky Plan...,Recent JWST observations serendipitously revea...,line,345.20,345.67,1,0.47,345.435,Finding t=0: Tracing the Origins of Rocky Plan...,finding t tracing the origins of rocky planete...,finding tracing origins rocky planetesimals jw...,find t trace the origin of rocky planetesimal ...,find trace origin rocky planetesimal jwst rece...
67431,2023.A.00009.S,Finding t=0: Tracing the Origins of Rocky Plan...,Recent JWST observations serendipitously revea...,line,345.55,346.02,1,0.47,345.785,Finding t=0: Tracing the Origins of Rocky Plan...,finding t tracing the origins of rocky planete...,finding tracing origins rocky planetesimals jw...,find t trace the origin of rocky planetesimal ...,find trace origin rocky planetesimal jwst rece...
67432,2023.A.00009.S,Finding t=0: Tracing the Origins of Rocky Plan...,Recent JWST observations serendipitously revea...,line,346.02,346.96,1,0.94,346.490,Finding t=0: Tracing the Origins of Rocky Plan...,finding t tracing the origins of rocky planete...,finding tracing origins rocky planetesimals jw...,find t trace the origin of rocky planetesimal ...,find trace origin rocky planetesimal jwst rece...
67433,2023.A.00009.S,Finding t=0: Tracing the Origins of Rocky Plan...,Recent JWST observations serendipitously revea...,line,346.85,347.79,1,0.94,347.320,Finding t=0: Tracing the Origins of Rocky Plan...,finding t tracing the origins of rocky planete...,finding tracing origins rocky planetesimals jw...,find t trace the origin of rocky planetesimal ...,find trace origin rocky planetesimal jwst rece...


In [94]:
measurements = measurements[measurements.diff_freq < 5]
lines = measurements.query('fs_type == "line"')
lines = lines[['project_code', 'diff_freq', 'med_freq']]
len(lines)

44230

In [95]:
len(sorted(lines['diff_freq'].round(2).unique()))

74

In [96]:
len(sorted(lines['med_freq'].round(2).unique()))

14800

In [97]:
len(lines)

44230

In [98]:
sorted(lines['diff_freq'].round(2).unique())

[0.05,
 0.06,
 0.07,
 0.08,
 0.09,
 0.1,
 0.11,
 0.12,
 0.13,
 0.14,
 0.15,
 0.17,
 0.18,
 0.23,
 0.24,
 0.25,
 0.26,
 0.27,
 0.28,
 0.29,
 0.3,
 0.39,
 0.46,
 0.47,
 0.48,
 0.49,
 0.5,
 0.51,
 0.52,
 0.53,
 0.59,
 0.63,
 0.93,
 0.94,
 0.95,
 0.96,
 0.97,
 0.98,
 0.99,
 1.0,
 1.01,
 1.02,
 1.03,
 1.05,
 1.06,
 1.43,
 1.86,
 1.87,
 1.88,
 1.89,
 1.9,
 1.91,
 1.92,
 1.93,
 1.94,
 1.97,
 1.98,
 1.99,
 2.0,
 2.01,
 2.02,
 2.03,
 2.04,
 2.05,
 2.06,
 2.07,
 2.08,
 2.09,
 2.1,
 2.12,
 2.13,
 2.14,
 2.15,
 3.89]

From the above, we can see that the greatest number of targets (when rounding to 2 decimal places) is 74 x 14800 = 1095200diffs = sorted(lines['diff_freq'].round(2).unique())
meds = sorted(lines['med_freq'].round(2).unique())

In [99]:
diffs = sorted(lines['diff_freq'].round(2).unique())
meds = sorted(lines['med_freq'].round(2).unique())

The functions below are to create the possible ranges based off desired windows.
- group_values_by_range: helper function to create ranges for both median frequency and frequency difference. starts based off lowest unique value in above lists, and only creates "ranges" for possible values (i.e doesn't create windows where no project has an observation)
- build_range: uses group_values_by_range for both median and difference in frequency and creates a list of lists, containing each "box" on the graph made in the EDA notebook.

In [100]:
def group_values_by_range(lst, group_size):
    lst.sort()  # Sort the list first
    ranges = []
    start = lst[0]  # Start with the minimum value

    while start < lst[-1]:
        end = start + group_size
        values_in_range = [val for val in lst if start <= val < end]
        ranges.append([start, end])
        
        # Move to the next value that's greater than the current end
        start = next((val for val in lst if val >= end), lst[-1] + 1)
        
    return ranges

def build_range(med_vals, diff_vals=0.2, remove_imp= True):
    '''
    Returns ranges with the range set by med_vals and diff_vals
    '''
    diff_range = group_values_by_range(diffs, diff_vals)
    med_range = group_values_by_range(meds, med_vals)
    
    ranges = []
    for item1 in diff_range:
        for item2 in med_range :
            ranges.append([item1, item2])
    
    return ranges

Below is a function to create a new table with new target values

In [101]:
def target_discretization(df, ranges):
    '''
    This function creates a new dataframe with project code and a new target variable vector,
    which will represent the target probabilities of a given project code
    to use for a machine learning model. It will discretize a large range of values (in this case, frequencies)
    into categories set in the ranges parameter.
    
    param df, pandas.dataframe: dataframe of measurements
    
    param ranges, 2- D array: Nested list of ranges to sort values into for each project. The length
    of the first dimension of this list will determine the length of the truth vector returned.
    [[[0.05, 0.25], [36.08, 41.08]],]
    '''
    df_new = pd.DataFrame(columns=['project_code', 'target']) # new df to return
    for pc in df.project_code.unique(): # loop through all line projects
        truth_vals = [0 for _ in range(len(ranges))] # create initial truth value list
        df_small = df[df['project_code'] == pc] # subset to correct observations
        for i in range(len(df_small)): # loop through all oobservations
            diff_f = df_small.iloc[i]['diff_freq']
            med_f = df_small.iloc[i]['med_freq']
            for a in range(len(ranges)): # Loop through ranges and match observation to range
                if diff_f >= ranges[a][0][0] and diff_f < ranges[a][0][1] and \
                med_f >= ranges[a][1][0] and med_f < ranges[a][1][1]:
                    truth_vals[a] = truth_vals[a] + 1 # add 1 for each observation in given range
        pos = sum(truth_vals) # now we change to probabilities
        for a in range(len(truth_vals)):
            if truth_vals[a] != 0:
                truth_vals[a] = truth_vals[a]/pos
        df_new.loc[len(df_new)] = [pc, truth_vals] # append to return dataframe
        
    return df_new

## Lets see if it worked!

In [102]:
target = target_discretization(lines, build_range(5))

In [103]:
target.iloc[0]['target'] 

[0,
 0,
 0,
 0.3,
 0.2,
 0.05,
 0.05,
 0,
 0.2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [104]:
sum(target.iloc[84]['target']) # sum to 1, good!

1.0

In [105]:
len(target.iloc[0]['target'])

854

In [106]:
lines.iloc[0]['diff_freq']

0.2400000000000091

In [107]:
len(build_range(5)) # yes! shape of target array should match number of targets

854

In [108]:
target['target']

0       [0, 0, 0, 0.3, 0.2, 0.05, 0.05, 0, 0.2, 0, 0, ...
1       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
3623    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3624    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3625    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3626    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3627    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: target, Length: 3628, dtype: object

## Lets check to see if it correctly created probabilities for a random project

In [109]:
target.iloc[1234]

project_code                                       2016.1.00854.S
target          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 1234, dtype: object

In [110]:
lines[lines['project_code'] == '2016.1.00854.S']

,project_code,diff_freq,med_freq
18518,2016.1.00854.S,1.87,228.435


In [111]:
target.iloc[1234]['target'].index(1)

640

In [112]:
build_range(5)[640]

[[1.86, 2.06], [226.76, 231.76]]

### TFIDF VECTORIZATION

In [113]:
import numpy as np
import pandas as pd
import textparser   # For potential use later
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [114]:
projects = pd.read_csv('nrao_projects.csv')
projects = projects.set_index('project_code')
projects.head()

,project_title,project_abstract,fs_type,target,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text
project_code,,,,,,,,,
2018.1.01205.L,Fifty AU STudy of the chemistry in the disk/en...,The huge variety of planetary systems discover...,line,1,Fifty AU STudy of the chemistry in the disk/en...,fifty au study of the chemistry in the disk en...,fifty au study chemistry disk envelope system ...,fifty au study of the chemistry in the disk en...,fifty au study chemistry disk envelope system ...
2022.1.00316.L,COMPASS: Complex Organic Molecules in Protosta...,The emergence of complex organic molecules in ...,line,1,COMPASS: Complex Organic Molecules in Protosta...,compass complex organic molecules in protostar...,compass complex organic molecules protostars s...,compass complex organic molecule in protostars...,compass complex organic molecule protostars sp...
2017.1.00161.L,ALCHEMI: the ALMA Comprehensive High-resolutio...,A great variety in gas composition is observed...,line,1,ALCHEMI: the ALMA Comprehensive High-resolutio...,alchemi the comprehensive high resolution extr...,alchemi comprehensive high resolution extragal...,alchemi the comprehensive high resolution extr...,alchemi comprehensive high resolution extragal...
2021.1.01616.L,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,line,1,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,jelly survey of nearby jellyfish and ram press...,jelly survey nearby jellyfish ram pressure str...,jelly survey of nearby jellyfish and ram press...,jelly survey nearby jellyfish ram pressure str...
2021.1.00869.L,Bulge symmetry or not? The hidden dynamics of ...,A radio survey of red giant SiO sources in the...,line,1,Bulge symmetry or not? The hidden dynamics of ...,bulge symmetry or not the hidden dynamics of t...,bulge symmetry hidden dynamics far side radio ...,bulge symmetry or not the hidden dynamic of th...,bulge symmetry hidden dynamic far side radio s...


In [115]:
line_projects = projects[projects['fs_type'] == 'line']
line_projects.shape

(3628, 9)

In [116]:
line_projects = line_projects[['lemmatized_no_sw_text']]
line_projects.head()

,lemmatized_no_sw_text
project_code,
2018.1.01205.L,fifty au study chemistry disk envelope system ...
2022.1.00316.L,compass complex organic molecule protostars sp...
2017.1.00161.L,alchemi comprehensive high resolution extragal...
2021.1.01616.L,jelly survey nearby jellyfish ram pressure str...
2021.1.00869.L,bulge symmetry hidden dynamic far side radio s...


In [117]:
# Generate count vectorized data frame
count_vectorizer = CountVectorizer()
cv_projects = count_vectorizer.fit_transform(line_projects.lemmatized_no_sw_text)
count_vectorizer.get_feature_names_out()

array(['aa', 'aalto', 'ab', 'abandance', 'abandunce', 'abc', 'abell',
       'aberrant', 'ability', 'ablate', 'ablation', 'ablative', 'able',
       'abmag', 'abnormally', 'aboard', 'abode', 'aborption', 'abound',
       'abrupt', 'abruptely', 'abruptly', 'absence', 'absent', 'absolute',
       'absolutely', 'absorb', 'absorbed', 'absorber', 'absorbing',
       'absorption', 'absortion', 'abstract', 'abundace', 'abundance',
       'abundances', 'abundant', 'abundantly', 'abut', 'ac', 'aca',
       'acccessible', 'accelerate', 'accelerated', 'acceleratedmaterial',
       'accelerates', 'acceleration', 'accelerations', 'accelerator',
       'accept', 'accepted', 'acces', 'access', 'accessibe',
       'accessibility', 'accessible', 'accidentally', 'accociated',
       'accommodate', 'accompany', 'accomplish', 'accord', 'accordingly',
       'account', 'accountable', 'accounting', 'accreation', 'accrection',
       'accrete', 'accretes', 'accreting', 'accretion', 'accretional',
       'acc

In [118]:
cv_projects

<3628x8641 sparse matrix of type '<class 'numpy.int64'>'
	with 281754 stored elements in Compressed Sparse Row format>

In [119]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_projects = tfidf_vectorizer.fit_transform(line_projects.lemmatized_no_sw_text).toarray()
#tfidf_vectorizer.get_feature_names_out()

In [121]:
tfidf_projects

array(...)

In [122]:
tfidf_vectorizer

TfidfVectorizer()

In [ ]:
#pip install tensorflow scikit-learn

## DATA PREPARATION

In [ ]:
# tfidf_projects

In [123]:
len(target['target'].iloc[33])

854

In [124]:
print(target.iloc[1])

project_code                                       2011.0.00017.S
target          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 1, dtype: object


In [ ]:
# from sklearn.model_selection import train_test_split
# from scipy.sparse import csr_matrix
# import numpy as np

# X = csr_matrix(tfidf_projects)  # Assuming this is your TF-IDF features matrix
# y = np.stack(target['target'].values)  # Converting target dataframe column to a numpy matrix

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#X[1]

In [ ]:
#print(y)

In [ ]:
#X_train

In [ ]:
#y_train

In [ ]:
#X_test

In [ ]:
#y_test

## MODELS

In [ ]:
#pip install torch

In [ ]:
#y = target['target'].values
#y.size

In [ ]:
#type(y)

In [ ]:
#type(X)

In [ ]:
# #PREPARE DATA
# import torch
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.model_selection import train_test_split
# from scipy.sparse import csr_matrix
# import numpy as np

# # Your provided code for splitting the dataset
# X = tfidf_projects  # Assuming this is your TF-IDF features matrix
# y = target['target'].values


# threshold = 0.2

# y = pd.Series(y)

# # Apply thresholding within each list in the series
# y_thresholded = y.apply(lambda x: [int(item >= threshold) for item in x])
# y = np.array(y_thresholded)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Convert the sparse matrix to dense, then to PyTorch tensors
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
# y_train_array = np.array(y_train.tolist(), dtype=np.float32)
# y_test_array = np.array(y_test.tolist(), dtype=np.float32)

# # Then convert to PyTorch tensors
# y_train_tensor = torch.tensor(y_train_array, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test_array, dtype=torch.float32)

# # Create DataLoader instances
# train_data = TensorDataset(X_train_tensor, y_train_tensor)
# test_data = TensorDataset(X_test_tensor, y_test_tensor)

# batch_size = 32
# train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
# test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
# # DEFINE NN MODEL FOR TEXT CLASSIFICATION, NETWORK USES FULLY CONNECTED LAYERS 
# # W/RELU ACTIVATIONS & DROPOUT FOR REGULARIZATION 
# import torch.nn as nn
# import torch.nn.functional as F

# class TextClassifier(nn.Module):
#     def __init__(self, input_dim, num_classes):
#         super(TextClassifier, self).__init__()
#         # First fully connected layer
#         self.fc1 = nn.Linear(input_dim, 512)
#         # Dropout layer for regularization
#         self.dropout = nn.Dropout(0.5)
#         # Second fully connected layer that outputs our desired number of classes
#         self.fc2 = nn.Linear(512, num_classes)
        
#     def forward(self, x):
#         # Input passes through the first layer, then a ReLU activation function
#         x = F.relu(self.fc1(x))
#         # Apply dropout
#         x = self.dropout(x)
#         # Pass through the second layer
#         x = self.fc2(x)
#         # Apply sigmoid activation function to output probabilities
#         x = torch.sigmoid(x)
#         return x

In [ ]:
# import torch.optim as optim

# # Correctly get the input dimension from the TF-IDF feature matrix
# input_dim = X_train.shape[1]
# # Correctly get the number of classes from the shape of y_train_numeric
# num_classes = y_train_array.shape[1]  # Use y_train_numeric to get the correct shape

# model = TextClassifier(input_dim, num_classes)

# # Use GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Binary Cross-Entropy Loss for multi-label classification
# loss_fn = nn.BCELoss()
# # Adam optimizer
# optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
# # TRAIN MODEL

# def train_model(model, dataloader, loss_fn, optimizer):
#     model.train()
#     total_loss = 0
#     for batch, (X, y) in enumerate(dataloader):
#         X, y = X.to(device), y.to(device)
        
#         # Forward pass: Compute predicted y
#         pred = model(X)
#         # Compute loss
#         loss = loss_fn(pred, y)
        
#         # Zero gradients, perform a backward pass, and update the weights.
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         total_loss += loss.item()
    
#     avg_loss = total_loss / len(dataloader)
#     print(f"Average Training Loss: {avg_loss:.4f}")

In [ ]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# def evaluate_model(model, dataloader, threshold=0.5):
#     model.eval()
#     all_preds = []
#     all_targets = []

#     with torch.no_grad():
#         for X, y in dataloader:
#             X, y = X.to(device), y.to(device)
#             preds = model(X)
            
#             # Apply threshold to convert probabilities to binary predictions
#             binary_preds = (preds >= threshold).float()
            
#             all_preds.append(binary_preds.cpu())
#             all_targets.append(y.cpu())

#     # Concatenate all batch results
#     all_preds = torch.cat(all_preds, dim=0).numpy()
#     all_targets = torch.cat(all_targets, dim=0).numpy()

#     # Ensure binary format (multilabel-indicator)
#     all_preds = np.round(all_preds).astype(int)
#     all_targets = all_targets.astype(int)

#     # Calculate metrics
#     accuracy = accuracy_score(all_targets, all_preds)
#     precision = precision_score(all_targets, all_preds, average='macro', zero_division=0)
#     recall = recall_score(all_targets, all_preds, average='macro', zero_division=0)
#     f1 = f1_score(all_targets, all_preds, average='macro', zero_division=0)

#     print(f"Accuracy: {accuracy:.4f}")
#     print(f"Precision: {precision:.4f}")
#     print(f"Recall: {recall:.4f}")
#     print(f"F1 Score: {f1:.4f}")
    
#     # Return both predictions and targets
#     return all_preds, all_targets

In [ ]:
# predictions, targets = evaluate_model(model, test_dataloader, threshold=0.5)

# # Now you can print or inspect predictions and targets
# print(predictions)
# print(targets)

In [ ]:
# # Example: Train the model for one epoch (or more) before evaluating
# train_model(model, train_dataloader, loss_fn, optimizer)

# # Evaluate the model
# all_preds, all_targets = evaluate_model(model, test_dataloader, loss_fn)

In [ ]:
#import pandas as pd

#threshold = 0.2

# Assuming 'y' is your series containing lists
#y = pd.Series(y)

# Apply thresholding within each list in the series
#y_thresholded = y.apply(lambda x: [int(item >= threshold) for item in x])


In [ ]:
# all_preds

In [ ]:
# def predict(model, dataloader, threshold=0.5):
#     model.eval()  # Set the model to evaluation mode
#     predictions = []
#     true_labels = []

#     with torch.no_grad():  # No need to track gradients
#         for X, y in dataloader:
#             X = X.to(device)
#             preds = model(X)
#             preds_binary = (preds >= threshold).float()  # Convert to binary predictions
#             predictions.extend(preds_binary.cpu().numpy())
#             true_labels.extend(y.cpu().numpy())

#     return np.array(predictions), np.array(true_labels)

# # Using the function to predict on the test set
# predictions, true_labels = predict(model, test_dataloader)

# # Optionally, you can evaluate the predictions to see how well your model is performing
# accuracy = accuracy_score(true_labels, predictions)
# precision = precision_score(true_labels, predictions, average='macro', zero_division=0)
# recall = recall_score(true_labels, predictions, average='macro', zero_division=0)
# f1 = f1_score(true_labels, predictions, average='macro', zero_division=0)

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")


In [ ]:
# def train_loop(dataloader, model, loss_fn, optimizer, epochs=10):
#     for epoch in range(epochs):
#         model.train()  # Set the model to training mode
#         total_loss = 0
#         for batch, (X, y) in enumerate(dataloader):
#             X, y = X.to(device), y.to(device)

#             # Compute prediction and loss
#             pred = model(X)
#             loss = loss_fn(pred, y)

#             # Backpropagation
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             total_loss += loss.item()

#         print(f"Epoch {epoch+1}\n-------------------------------")
#         print(f"Train loss: {total_loss/len(dataloader):.4f}")

# def test_loop(dataloader, model, loss_fn):
#     model.eval()  # Set the model to evaluation mode
#     total_loss = 0
#     with torch.no_grad():
#         for X, y in dataloader:
#             X, y = X.to(device), y.to(device)
#             pred = model(X)
#             total_loss += loss_fn(pred, y).item()

#     print("Test\n-------------------------------")
#     print(f"Test loss: {total_loss/len(dataloader):.4f}\n")

# # Example usage:
# train_loop(train_dataloader, model, loss_fn, optimizer)
# test_loop(test_dataloader, model, loss_fn)

In [ ]:
# import matplotlib.pyplot as plt

# # Assume these lists are filled within the train_loop and test_loop
# train_losses = []
# test_losses = []

# # Call train_loop and test_loop here to fill the above lists
# train_loop(train_dataloader, model, loss_fn, optimizer)
# test_loop(test_dataloader, model, loss_fn)

# # Plotting
# plt.figure(figsize=(10, 6))
# plt.plot(train_losses, label='Training Loss')
# plt.plot(test_losses, label='Test Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Test Loss Over Epochs')
# plt.legend()
# plt.show()

In [ ]:
#import numpy as np
#print(np.__version__)

In [ ]:
#pip install numpy==1.19.2

In [125]:
import numpy as np
import sys

# Assuming X is your NumPy array
np.set_printoptions(threshold=sys.maxsize)

# Now, when you print the array, it should display in full
#print(X[0])

In [126]:
y_train.shape[1]

854

In [129]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l1, l2, l1_l2
import numpy as np

np.random.seed(42)

tf.random.set_seed(42)

# Assuming you have your data ready in X and y

# Check if tfidf_projects is a sparse matrix and convert it to a NumPy array if so
if hasattr(tfidf_projects, "toarray"):
    X = np.array(tfidf_projects.toarray())
else:
    X = np.array(tfidf_projects)  # Assuming tfidf_projects is already a dense array

# Convert your target to a suitable format
y = np.stack(target['target'].values)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CNN model
model_cnn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, 
                          activation='relu', 
                          input_shape=(X_train.shape[1],),
                          kernel_regularizer=l2(0.001)),  # Applying L2 regularization
    tf.keras.layers.Reshape((1024, 1)),
    tf.keras.layers.Conv1D(128, kernel_size=5, 
                           activation='relu', 
                           padding='same',
                           kernel_regularizer=l2(0.001)),  # Increased filters and applied L2 regularization
    tf.keras.layers.MaxPooling1D(pool_size=5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, 
                          activation='relu', 
                          kernel_regularizer=l2(0.001)),  # Increased nodes and applied L2 regularization
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model
model_cnn.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Fit the model
model_cnn.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 14s 330ms/step - accuracy: 0.0225 - loss: 0.9090 - val_accuracy: 0.0516 - val_loss: 0.0599
Epoch 2/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 328ms/step - accuracy: 0.0384 - loss: 0.0997 - val_accuracy: 0.0929 - val_loss: 0.0166
Epoch 3/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 13s 341ms/step - accuracy: 0.0500 - loss: 0.0133 - val_accuracy: 0.0516 - val_loss: 0.0101
Epoch 4/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 13s 354ms/step - accuracy: 0.0484 - loss: 0.0104 - val_accuracy: 0.0929 - val_loss: 0.1700
Epoch 5/5
37/37 ━━━━━━━━━━━━━━━━━━━━ 13s 360ms/step - accuracy: 0.0415 - loss: 0.0604 - val_accuracy: 0.0516 - val_loss: 0.0097


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming you have your data ready in X and y
# Check if tfidf_projects is a sparse matrix and convert it to a NumPy array if so
if hasattr(tfidf_projects, "toarray"):
    X = np.array(tfidf_projects.toarray())
else:
    X = np.array(tfidf_projects) # Assuming tfidf_projects is already a dense array

# Convert your target to a suitable format
y = np.stack(target['target'].values)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.conv1d = nn.Conv1d(1, 128, kernel_size=5, padding=2)
        self.maxpool1d = nn.MaxPool1d(kernel_size=5)
        self.flatten = nn.Flatten()
        self.fc2 = nn.Linear(128 * (1024 // 5), 512)
        self.dropout = nn.Dropout(0.5)
        self.fc3 = nn.Linear(512, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = x.unsqueeze(1)
        x = F.relu(self.conv1d(x))
        x = self.maxpool1d(x)
        x = self.flatten(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return torch.sigmoid(x)

# Initialize the model
model_cnn = CNN(X_train.shape[1], y_train.shape[1])

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_cnn.parameters(), lr=0.005, weight_decay=0.001)

# Train the model
num_epochs = 3
batch_size = 64

for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_acc = 0.0
    num_samples = 0

    for i in range(0, X_train.shape[0], batch_size):
        inputs = X_train[i:i+batch_size]
        labels = y_train[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model_cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * inputs.size(0)

        # Calculate accuracy
        predicted = (outputs > 0.5).float()
        correct = (predicted == labels).float().sum()
        epoch_acc += correct.item()
        num_samples += labels.size(0)
        print(outputs.shape)
        
'''
    epoch_loss /= num_samples
    epoch_acc /= num_samples

    # Print the loss and accuracy for every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
'''

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
#pip install transformers

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming you have your data ready in X and y
# Check if tfidf_projects is a sparse matrix and convert it to a NumPy array if so
if hasattr(tfidf_projects, "toarray"):
    X = np.array(tfidf_projects.toarray())
else:
    X = np.array(tfidf_projects) # Assuming tfidf_projects is already a dense array

# Convert your target to a suitable format
y = np.stack(target['target'].values)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

# Load the pre-trained BERT model and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the BERT-based model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, output_size):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(bert_model.config.hidden_size, output_size)

    def forward(self, input_ids, attention_mask=None):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.fc(pooled_output)
        return torch.sigmoid(x)

# Initialize the model
output_size = y_train.shape[1]
model = BERTClassifier(bert_model, output_size)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

# Train the model
num_epochs = 3
batch_size = 64

for epoch in range(num_epochs):
    epoch_loss = 0.0
    num_samples = 0

    for i in range(0, X_train.shape[0], batch_size):
        inputs = X_train[i:i+batch_size]
        labels = y_train[i:i+batch_size]

        # Tokenize and convert the input data to tensor
        input_ids = torch.tensor([tokenizer.encode(str(inp), add_special_tokens=True, max_length=512, pad_to_max_length=True, truncation=True) for inp in inputs])
        attention_mask = (input_ids != 0).float()

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * inputs.size(0)
        num_samples += labels.size(0)

    epoch_loss /= num_samples

    # Print the loss for every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

In [ ]:
#pip install "tensorflow-probability[tf]"

In [ ]:
#pip install --upgrade numpy pymc

In [ ]:
X = np.array(tfidf_projects)

In [ ]:
import pymc as pm
import numpy as np
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Building a simple Bayesian Linear Regression Model
with pm.Model() as model:
    # Priors for unknown model parameters
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10, shape=X_train.shape[1])
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Expected value of outcome (Linear model)
    mu = alpha + pm.math.dot(X_train, beta)

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=y_train)

    # Posterior distribution
    trace = pm.sample(1000, return_inferencedata=False)

# To visualize the posterior distribution
pm.traceplot(trace)

# To get the summary of the posterior distribution
summary = pm.summary(trace)
print(summary)


In [ ]:
#model_rnn = tf.keras.models.Sequential([
#    tf.keras.layers.Embedding(input_dim=X_train.shape[1], output_dim=64, input_length=X_train.shape[1]),
#    tf.keras.layers.SimpleRNN(128, return_sequences=True),
#    tf.keras.layers.SimpleRNN(128),
#    tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid')
#])#
#
#model_rnn.compile(optimizer='adam',
#                  loss='binary_crossentropy',
#                  metrics=['accuracy'])#
#
#model_rnn.fit(X_train.toarray(), y_train, epochs=10, batch_size=32, validation_split=0.1)  # Note: RNN requires dense input

In [ ]:
python -m venv pymc-venv
source pymc-venv/bin/activate  # Use `.\pymc-venv\Scripts\activate` on Windows
pip install numpy pymc
